# Import

In [ ]:
import torch 
import random
import numpy as np
import os
from torchvision.transforms import v2

import torch

from torch.utils.data import Dataset, DataLoader, random_split

import numpy as np
import pandas as pd
from PIL import Image

import os
import random
from tqdm.notebook import tqdm    

In [ ]:
%pip install efficientnet_pytorch

from efficientnet_pytorch import EfficientNet

In [ ]:
%pip install ttach

import ttach as tta

# Setting

In [ ]:
test_name = "cp3_231208_1556"
test_epoch = 7
test_model_path = '/kaggle/input/trained-model'
test_dataset_path = '/kaggle/input/image-classification2023-2nd/test'

In [ ]:
seed = 42
test_batch_size = 256
epoch_num = 5

In [ ]:
test_transforms = v2.Compose([
    v2.CenterCrop((240, 240)),
    # v2.Lambda(lambda img:v2.functional.adjust_brightness(img, 1.2)),
    v2.RandomRotation(degrees=(0, 30)), # 경진대회 종료 후 확인
    v2.Lambda(lambda img:v2.functional.adjust_contrast(img, 1.2)),
    v2.Lambda(lambda img:v2.functional.adjust_saturation(img, 1.2)),
    v2.ToTensor(),
])

# Test

In [ ]:
directories = [
    '/kaggle/working/csv',
    '/kaggle/working/save_state_dict',
    '/kaggle/working/save_dict',
]

for dir_path in directories:
    os.makedirs(dir_path, exist_ok=True)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(seed)

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# 제너레이터 시드값 고정
g = torch.Generator()
g.manual_seed(seed)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b1', 
                                    num_classes=2) 

# 장비 할당
model = model.to(device)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.df.iloc[idx, 0] 
        image_path = f'/kaggle/input/image-classification2023-2nd/test/{image_name}'

        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image

In [ ]:
test_df = pd.read_csv('/kaggle/input/image-classification2023-2nd/sample_submission.csv')

datasets_test = CustomDataset(test_df, transform=test_transforms)

loader_test = DataLoader(dataset=datasets_test, batch_size=test_batch_size, 
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=0)

model.eval()

In [ ]:
# TTA 변환 정의
tta_transforms = tta.Compose([
    tta.HorizontalFlip(),
    tta.Rotate90(angles=[0, 90, 180, 270]),
    tta.Multiply(factors=[0.9, 1, 1.1]),   
])

In [ ]:
# 모델 가중치 불러오기
model.load_state_dict(torch.load(f'/kaggle/working/save_dict/{test_name}_{str(test_epoch)}.pth'))

# TTA 모델 생성
tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)

predictions = []

for images in tqdm(loader_test):  # 레이블은 사용하지 않음
    images = images.to(device)  # 장비 할당
    with torch.no_grad():
        outputs = tta_model(images)
        prediction = outputs.argmax(dim=1).cpu().numpy()
        predictions.extend(prediction)
        
test_df['label'] = predictions

test_df.to_csv(f'/kagle/working/csv/{test_name}_{test_epoch}.csv', index=False)